In [ ]:
gcloud auth list

In [ ]:
gcloud config list project

##Task 1. Run a simple Dataflow job

In [ ]:
DATASET_NAME=lab_594
BUCKET_NAME=qwiklabs-gcp-00-35f44ccb1d23-marking
REGION=us-east1
TABLE_NAME=customers_871

In [ ]:
PROJECT_ID=$(gcloud config get-value project)
PROJECT_NUMBER=$(gcloud projects describe "$PROJECT_ID" --format="json" | jq -r '.projectNumber')
echo $PROJECT_NUMBER

In [ ]:
gcloud services enable apikeys.googleapis.com

In [ ]:
gcloud alpha services api-keys create --display-name="awesome"

In [ ]:
KEY_NAME=$(gcloud alpha services api-keys list --format="value(name)" --filter "displayName=awesome")

In [ ]:
API_KEY=$(gcloud alpha services api-keys get-key-string $KEY_NAME --format="value(keyString)")

In [ ]:
bq mk $DATASET_NAME

In [ ]:
gsutil mb gs://$BUCKET_NAME

In [ ]:
gsutil cp gs://cloud-training/gsp323/lab.csv  .

In [ ]:
gsutil cp gs://cloud-training/gsp323/lab.schema .

In [ ]:
cat lab.schema

##Task 2. Run a simple Dataproc job

In [ ]:
bq mk --table $DATASET_NAME.$TABLE_NAME lab.schema

In [ ]:
gcloud dataflow jobs run quicklab-jobs --gcs-location gs://dataflow-templates-$REGION/latest/GCS_Text_to_BigQuery --region $REGION --worker-machine-type e2-standard-2 --staging-location gs://$DEVSHELL_PROJECT_ID-marking/temp --parameters inputFilePattern=gs://cloud-training/gsp323/lab.csv,JSONPath=gs://cloud-training/gsp323/lab.schema,outputTable=$DEVSHELL_PROJECT_ID:$DATASET_NAME.$TABLE_NAME,bigQueryLoadingTemporaryDirectory=gs://$DEVSHELL_PROJECT_ID-marking/bigquery_temp,javascriptTextTransformGcsPath=gs://cloud-training/gsp323/lab.js,javascriptTextTransformFunctionName=transform


##Task 3. Use the Google Cloud Speech-to-Text API

In [ ]:
TASK_3_BUCKET_NAME=gs://qwiklabs-gcp-00-35f44ccb1d23-marking/task3-gcs-810.result

In [ ]:
gcloud projects add-iam-policy-binding $DEVSHELL_PROJECT_ID \
    --member "serviceAccount:$PROJECT_NUMBER-compute@developer.gserviceaccount.com" \
    --role "roles/storage.admin"

In [ ]:
gcloud projects add-iam-policy-binding $DEVSHELL_PROJECT_ID \
  --member=user:$USER_EMAIL \
  --role=roles/dataproc.editor

In [ ]:
gcloud projects add-iam-policy-binding $DEVSHELL_PROJECT_ID \
  --member=user:$USER_EMAIL \
  --role=roles/storage.objectViewer

In [ ]:
gcloud compute networks subnets update default \
    --region $REGION \
    --enable-private-ip-google-access

In [ ]:
gcloud iam service-accounts create quicklab \
  --display-name "my natural language service account"

In [ ]:
gcloud iam service-accounts keys create ~/key.json \
  --iam-account quicklab@${GOOGLE_CLOUD_PROJECT}.iam.gserviceaccount.com

In [ ]:
export GOOGLE_APPLICATION_CREDENTIALS="/home/$USER/key.json"

In [ ]:
gcloud auth activate-service-account quicklab@${GOOGLE_CLOUD_PROJECT}.iam.gserviceaccount.com --key-file=$GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
gcloud ml language analyze-entities --content="Old Norse texts portray Odin as one-eyed and long-bearded, frequently wielding a spear named Gungnir and wearing a cloak and a broad hat." > result.json


##Task 4. Use the Cloud Natural Language API

In [ ]:
TASK_4_BUCKET_NAME=gs://qwiklabs-gcp-00-35f44ccb1d23-marking/task4-cnl-361.result

In [ ]:
gsutil cp result.json $TASK_4_BUCKET_NAME

In [ ]:
cat > request.json <<EOF
{
  "config": {
      "encoding":"FLAC",
      "languageCode": "en-US"
  },
  "audio": {
      "uri":"gs://cloud-training/gsp323/task3.flac"
  }
}
EOF

In [ ]:
curl -s -X POST -H "Content-Type: application/json" --data-binary @request.json \
"https://speech.googleapis.com/v1/speech:recognize?key=${API_KEY}" > result.json

In [ ]:
gsutil cp result.json $TASK_3_BUCKET_NAME

In [ ]:
gcloud iam service-accounts create quickstart

In [ ]:
gcloud iam service-accounts keys create key.json --iam-account quickstart@${GOOGLE_CLOUD_PROJECT}.iam.gserviceaccount.com


In [ ]:
gcloud auth activate-service-account --key-file key.json

In [ ]:
export ACCESS_TOKEN=$(gcloud auth print-access-token)

In [ ]:
cat > request.json <<EOF
{
   "inputUri":"gs://spls/gsp154/video/train.mp4",
   "features": [
       "TEXT_DETECTION"
   ]
}
EOF

In [ ]:
curl -s -H 'Content-Type: application/json' \
    -H "Authorization: Bearer $ACCESS_TOKEN" \
    'https://videointelligence.googleapis.com/v1/videos:annotate' \
    -d @request.json

In [ ]:
curl -s -H 'Content-Type: application/json' -H "Authorization: Bearer $ACCESS_TOKEN" 'https://videointelligence.googleapis.com/v1/operations/OPERATION_FROM_PREVIOUS_REQUEST' > result1.json


In [ ]:
curl -s -X POST -H "Content-Type: application/json" --data-binary @request.json \
"https://speech.googleapis.com/v1/speech:recognize?key=${API_KEY}" > result.json

In [ ]:
curl -s -H 'Content-Type: application/json' \
    -H "Authorization: Bearer $ACCESS_TOKEN" \
    'https://videointelligence.googleapis.com/v1/videos:annotate' \
    -d @request.json

In [ ]:
curl -s -H 'Content-Type: application/json' -H "Authorization: Bearer $ACCESS_TOKEN" 'https://videointelligence.googleapis.com/v1/operations/OPERATION_FROM_PREVIOUS_REQUEST' > result1.json